In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train400/test_398.png
/kaggle/input/train400/test_032.png
/kaggle/input/train400/test_173.png
/kaggle/input/train400/test_277.png
/kaggle/input/train400/test_021.png
/kaggle/input/train400/test_116.png
/kaggle/input/train400/test_360.png
/kaggle/input/train400/test_095.png
/kaggle/input/train400/test_334.png
/kaggle/input/train400/test_102.png
/kaggle/input/train400/test_304.png
/kaggle/input/train400/test_354.png
/kaggle/input/train400/test_127.png
/kaggle/input/train400/test_015.png
/kaggle/input/train400/test_093.png
/kaggle/input/train400/test_126.png
/kaggle/input/train400/test_100.png
/kaggle/input/train400/test_108.png
/kaggle/input/train400/test_014.png
/kaggle/input/train400/test_267.png
/kaggle/input/train400/test_130.png
/kaggle/input/train400/test_166.png
/kaggle/input/train400/test_055.png
/kaggle/input/train400/test_141.png
/kaggle/input/train400/test_275.png
/kaggle/input/train400/test_345.png
/kaggle/input/train400/test_215.png
/kaggle/input/train400/test_

In [3]:
import torch
from torch import nn

# Define Model

In [5]:
class DnCNN(nn.Module):
    def __init__(self, num_layers, num_filters=64, image_channels=1):
        super(DnCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=num_filters, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        
        layers = []
        for _ in range(num_layers - 2):
            layers.append(nn.Conv2d(in_channels=num_filters, out_channels=num_filters, kernel_size=3, padding=1))
            layers.append(nn.BatchNorm2d(num_filters))
            layers.append(nn.ReLU(inplace=True))
            
        self.middle_layers = nn.Sequential(*layers)
        
        self.final = nn.Conv2d(in_channels=num_filters, out_channels=image_channels, kernel_size=3, padding=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.middle_layers(x)
        x = self.final(x)
        return x

In [6]:
test = torch.rand(1, 1, 64, 64)
model = DnCNN(10)
model.forward(test)

tensor([[[[-0.0227,  0.1482, -0.1257,  ...,  0.5114,  0.3591,  0.1933],
          [ 0.2910,  0.0830,  1.0561,  ...,  0.1852,  0.4037,  0.1759],
          [-0.0569, -0.0480,  0.5780,  ...,  0.2078,  0.3073, -0.4408],
          ...,
          [ 0.1020,  0.7258,  0.8128,  ..., -0.0319, -0.1405,  0.2226],
          [ 0.6977,  0.7132, -0.4239,  ...,  0.0635,  0.3275,  0.3368],
          [ 0.2555,  0.0662, -0.2489,  ..., -0.1624, -0.7348, -0.4003]]]],
       grad_fn=<ConvolutionBackward0>)

In [8]:
model(test).shape

torch.Size([1, 1, 64, 64])

# Create Dataset

In [19]:
from PIL import Image
import torchvision.transforms as transforms

transform = transforms.ToTensor()
folder_path = '/kaggle/input/train400'

train_y = []
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for file in files:
    file_path = os.path.join(folder_path, file)
    image = Image.open(file_path).convert('L')
    image_tensor = transform(image)
    train_y.append(image_tensor)
    
train_x = []
for image_tensor in train_y:
    noise = torch.randn(image_tensor.shape) * (25/255)
    image_tensor = image_tensor.clone() + noise
    image_tensor = torch.clamp(image_tensor, 0, 1)
    train_x.append(image_tensor)

In [22]:
print(train_y[:5])
print()
print(train_x[:5])

[tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 0.9961],
         [1.0000, 1.0000, 1.0000,  ..., 0.9882, 0.9961, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 0.9686, 0.9961, 0.9961],
         ...,
         [0.4549, 0.4980, 0.4941,  ..., 0.4627, 0.4706, 0.4431],
         [0.3686, 0.4118, 0.4157,  ..., 0.5098, 0.4902, 0.4471],
         [0.3882, 0.4000, 0.4118,  ..., 0.4980, 0.4941, 0.4902]]]), tensor([[[0.4863, 0.2431, 0.2000,  ..., 0.3686, 0.3333, 0.2980],
         [0.3373, 0.2275, 0.3490,  ..., 0.2471, 0.2235, 0.2667],
         [0.4510, 0.5529, 0.3098,  ..., 0.3216, 0.2392, 0.3490],
         ...,
         [0.2902, 0.2980, 0.2863,  ..., 0.2588, 0.2667, 0.2549],
         [0.2863, 0.2980, 0.3216,  ..., 0.2549, 0.2824, 0.2588],
         [0.2980, 0.3098, 0.3725,  ..., 0.2902, 0.3216, 0.2745]]]), tensor([[[0.2588, 0.2353, 0.2157,  ..., 0.0431, 0.0431, 0.0431],
         [0.2706, 0.2157, 0.2118,  ..., 0.0431, 0.0431, 0.0431],
         [0.2235, 0.2510, 0.2392,  ..., 0.0431, 0.0431,

In [14]:
MSELoss = torch.nn.MSELoss()

MSELoss()

In [ ]:
optimizer = torch.optim.Adam()
def train(epochs):
    for i in range(epochs):
        for item in ds:
            

# Training

In [ ]:
model.train()
for i in range(epochs):
    for item in ds: